<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_Salesforce_Generate_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy document using GetAccept *API*

Functions to copy document between entities and to create fillable PDF files from GetAccept templates or documents

In [0]:
# import dependencies
import requests

In [0]:
BASEURL = "https://api.getaccept.com/v1"
#@title Enter GetAccept login information
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
#@markdown Leave empty to use default entity or show list of available entities
ENTITY_ID = "" #@param {type: "string"}
#@markdown Find source document/template id by going to the document in GetAccept \
#@markdown and copy the id from the URL in the browser, eg. abc123de
LISTENER_URL = "" #@param {type: "string"}
EVENTS = ['document.sent','document.viewed','document.reviewed','document.opened','document.signed','document.approved','document.expired','document.rejected','document.partially_signed','document.recalled','document.hardbounced']

#@markdown Optional notification email to notify on errors
NOTIFICATION_EMAIL = "" #@param {type: "string"}

In [0]:
# Make auth request and store auth header
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": ENTITY_ID }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))


In [0]:
if LISTENER_URL != "":
  # Create listener webhooks
  for event_name in EVENTS:
    payload = {
      "event": event_name,
      "target_url": LISTENER_URL,
      "global": True,
      "notification_email": NOTIFICATION_EMAIL
    }
    response = requests.post(BASEURL+"/subscriptions",headers=source_auth_headers, json=payload)
    subscription_data = response.json()
    if 'id' in subscription_data:
      print(event_name+': Successfully created subscription')
    else:
      print(event_name+': '+ str(subscription_data) )